In [22]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
# import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
# import dask_xgboost as xgb
# import dask.dataframe as dd
from sklearn import preprocessing, metrics
from sklearn.preprocessing import LabelEncoder
import gc
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [38]:
os.listdir('../data/input')

['.DS_Store',
 'calendar.csv',
 'sell_prices.csv',
 'sales_train_validation.csv',
 'sample_submission.csv']

In [36]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def read_data():
    print('Reading files...')
    calendar = pd.read_csv('../data/input/calendar.csv')
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('../data/input/sell_prices.csv')
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv('../data/input/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv('../data/input/sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission


def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows=55000000, merge=False):
    
    # melt sales data, get it ready for training
    _ids = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    sales_train_validation = pd.melt(sales_train_validation, id_vars=_ids, var_name='day', value_name='demand')
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    d_col_1 = [f'd_{x}' for x in np.arange(1914, 1941+1, 1)]
    d_col_2 = [f'd_{x}' for x in np.arange(1942, 1969+1, 1)]
    d_col_1 = ['id'] + d_col_1
    d_col_2 = ['id'] + d_col_2
    
    test1.columns = d_col_1
    test2.columns = d_col_2
    del d_col_1, d_col_2
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    test1 = pd.melt(test1, id_vars=_ids, var_name='day', value_name='demand')
    test2 = pd.melt(test2, id_vars=_ids, var_name='day', value_name='demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis=0)
    
    del sales_train_validation, test1, test2
    gc.collect()
    
    # get only a sample for fst training
    if nrows is not None:
        data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace=True, axis=1)
    
    # delete test2 for now
#     data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how='left', left_on=['day'], right_on=['d'])
        data.drop(['d', 'day'], inplace=True, axis=1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data

In [37]:
%%time
calendar, sell_prices, sales_train_validation, submission = read_data()
df = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows=None, merge=True)

Reading files...
Calendar has 1969 rows and 14 columns
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to 3226.27 Mb (9.4% reduction)
Our final dataset to train has 60034810 rows and 18 columns
CPU times: user 2min 39s, sys: 2min 2s, total: 4min 42s
Wall time: 5min 24s


In [38]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN


In [39]:
df.shape

(60034810, 18)

In [40]:
df.isnull().sum()

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
demand                 0
part                   0
date                   0
wm_yr_wk               0
event_name_1    55095430
event_type_1    55095430
event_name_2    59882360
event_type_2    59882360
snap_CA                0
snap_TX                0
snap_WI                0
sell_price      12299413
dtype: int64

In [41]:
def preprocessing(df):
    
    # Date  ##########################################
    df['date'] = pd.to_datetime(df['date'].values)
    df['weekday'] = df['date'].dt.weekday
    
    # integrate 'snap' feature  ######################
    def snap(row):
        if 'CA' in row['store_id']:
            return row['snap_CA']
        elif 'TX' in row['store_id']:
            return row['snap_TX']
        elif 'WI' in row['store_id']:
            return row['snap_WI']
        else:
            pass

    df['snap'] = df.apply(snap, axis=1)
    
    # NaN  ############################################
    cols = {'event_name_1': 'Nodata', 
            'event_type_1': 'Nodata', 
            'event_name_2': 'Nodata', 
            'event_type_2': 'Nodata'}
    df.fillna(cols, inplace=True)
    
    # LabelEncoder  ####################################
    lbl_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for c in lbl_cols:
        lbl = LabelEncoder()
        df[c] = lbl.fit_transform(df[c].values)
        
    # Dtypes  ##########################################
    cat_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap', 'weekday']
    for c in cat_cols:
        df[c] = df[c].astype('category')
        
    return df

In [ ]:
%%time
df = preprocessing(df)

In [ ]:
class LGBMModel:
    
    def __init__(self, df, features):
        
        train = df[df['part'] == 'train']
        self.train_id = train['id'].values
        self.target = train['demand'].values
        
        validation = df[df['part'] == 'test1']
        self.val_id = validation['id'].values
        self.val_date = validation['date'].values
        
        evaluation = df[df['part'] == 'test2']
        self.eval_id = evalidation['id'].values
        self.eval_date = evalidation['date'].values
            
        self.features = features
        self.X = train[self.features].values
        self.vals = validation[self.features].values
        self.evals = evaluation[self.features].values
        
        del train, validation, evaluation
        gc.collect()
        
        self.importances = np.zeros((len(self.features)))
        self.importance_df = None
        self.best_score = 10000
        self.models = []

    def train(self, params, cv, num_boost_round=1000, early_stopping_rounds=20, verbose=200):
        print('LightGBM Model Training...')
        for i, (trn_idx, val_idx) in enumerate(cv.split(self.X)):

            train_data = lgb.Dataset(self.X[trn_idx], label=self.target[trn_idx])
            valid_data = lgb.Dataset(self.X[val_idx], label=self.target[val_idx], reference=train_data)

            model = lgb.train(params,
                              train_data,
                              valid_sets=[train_data, valid_data],
                              valid_names=['train', 'eval'],
                              num_boost_round=num_boost_round,
                              early_stopping_rounds=early_stopping_rounds,
                              verbose_eval=verbose
                              )
            self.models.append(model)

            self.importances += model.feature_importance() / cv.get_n_splits()

            pred = model.predict(self.X[val_idx], num_iteration=model.best_iteration)
            rmse = np.sqrt(mean_squared_error(y_true=self.target[val_idx], y_pred=pred))
            print(f'{i} Fold  RMSE: {rmse:.3f}')
            print('#'*30)

        self.importance_df = pd.DataFrame({
            'features': self.features,
            'importance': self.importances
        })

        return self.models, self.importance_df
    
    
    def evaluate(self):
        assert len(self.models) != 0, 'Model is not trained...'
        print('Evaluate...')
        
        pred_val = np.zeros(len(self.val_id))
        pred_eval = np.zeros(len(self.eval_id))
        
        for model in self.models:
            pred_val += model.predict(self.vals, num_iteration=model.best_iteration) / len(self.models)
            pred_eval += model.predict(self.evals, num_iteration=model.best_iteration) / len(self.models)
            
        res_val = pd.DataFrame({
            'id': self.val_id,
            'date': self.val_date,
            'pred': pred_val
        })
        
        res_val = pd.pivot(res_val, index='id', columns='date', values='demand').reset_index()
        
        res_eval = pd.DataFrame({
            'id': self.eval_id,
            'date': self.eval_date,
            'pred': pred_eval
        })
        
        res_eval = pd.pivot(res_eval, index='id', columns='date', values='demand').reset_index()
        
        F_list = [f'F{i+1}' for i in range(28)]
        
        res_val.columns = ['id'] + F_list
        res_eval.columns = ['id'] + F_list
        
        res = pd.concat([res_val, res_eval], axis=0)
        print('FINISH')
        
        return res
#         # For submit
#         print('Creating Submission')
#         res = pd.DataFrame()
#         F_list = [f'F{i}' for i in range(1, 29, 1)]
#         # Validation
#         id_list = res_val['id'].unique()
#         for _id in tqdm(id_list):
#             temp = res_val[res_val['id'] == _id].sort_values(by='date').reset_index(drop=True)
#             temp = temp[['pred']].T.reset_index(drop=True)
#             temp.columns = F_list
#             temp['id'] = _id
#             c = ['id'] + F_list
#             temp = temp[c]
#             res = pd.concat([res, temp], axis=0, ignore_index=True)
            
#         # Evaluation
#         id_list = res_eval['id'].unique()
#         for _id in tqdm(id_list):
#             temp = res_eval[res_eval['id'] == _id].sort_values(by='date').reset_index(drop=True)
#             temp = temp[['pred']].T.reset_index(drop=True)
#             temp.columns = F_list
#             temp['id'] = _id
#             c = ['id'] + F_list
#             temp = temp[c]
#             res = pd.concat([res, temp], axis=0, ignore_index=True)
    
    def visualize_feature_importance(self):
        _importance_df = self.importance_df.sort_values(by='importance', ascending=False)
        fig = plt.figure(figsize=(12, 8))
        sns.barplot(x='importance', y='features', data=_importance_df)
        plt.title('Feature Imporrance')
        plt.show()

In [ ]:
df.columns

In [ ]:
%%time
# config
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
}

cv = KFold(n_splits=3)
num_boost_round = 5000
early_stopping_rounds = 200
verbose = 1000
features = []

lgbm = LGBMModel(train, features)
model, importance_df = lgbm.train(params, cv, num_boost_round, early_stopping_rounds, verbose)

In [ ]:
res = lgbm.evaluate()